# **Reto I**

### 1. Datasets

Los datos de origen constan de dos archivos csv con la misma estructura y tipo de columnas.

* **trade_details**: dataset original con datos reales de operaciones financieras.
* **trade_details_snapshot**: copia de seguridad por posibles perdidas de datos.

### 2. Columnas y significado:

* **mfamily**: indica la familia de operaciones a la que pertenece.
* **mgroup**: indica el grupo de operaciones dentro de la familia.
* **mtype**: indica el tipo de operación dentro del grupo.
* **origin_trade_number**: indica el número de la operación de trading (la misma operación puede tener varios números de trading).
* **origin_contract_number**: indica el número de contrato de la operación (igual para todas las operaciones que pertenecen al mismo contrato).
* **maturity**: fecha de finalización del contrato de cada operación.

### 3. Descripción del problema:

En estos datasets se encuentran varias operaciones financieras de distinto tipo, que diferenciaremos mediante los distintos valores de las columnas mfamily, mgroup y mtype.

Existe un cierto tipo de operaciones especiales, llamadas FXSwaps. Estas pueden ser diferenciadas por medio de los siguientes valores:

**mfamily = CURR** \
**mgroup = FXD** \
**mtype = SWLEG**

Podemos ver en nuestro dataset que estas operaciones aparecen duplicadas, es decir, con el mismo **origin_contract_number** aunque distinto **origin_trade_number**. De estas operaciones duplicadas en origen, queremos obtener solo una de ellas.

La forma para decidir cuál de las operaciones nos interesa obtener es mediante la columna *maturity*. De ambas operaciones de trading (distinto origin_trade_number) para un mismo contrato (origin_contract_number), queremos obtener solo la *long leg*, es decir, la que tiene una mayor fecha de vencimiento (fecha más actual de la columna maturity).

Existe un cierto problema en nuestro dataset trade_details que tendremos que solucionar. Podemos ver que para algunas operaciones el campo maturity vendrá como *null*, es decir, sin informar. En estos casos, deberemos buscar esa operacion en el dataset trade_details_snapshot y el respectivo campo maturity para poder saber cuál de las dos operaciones es la *long leg* y filtrar la *short leg* 

**NOTA: Si se quiere conocer más el significado de estas operaciones financieras: https://es.wikipedia.org/wiki/Swap_(finanzas)**

### 4. Reto:

* Obtener un dataframe final donde tengamos todas las operaciones originales excepto los short leg de los contratos tipo FXSwap.
* Aunque usemos el valor de la columna maturity del dataset trade_details_snapshot en los casos que venga en la trade_details a *null*, en el dataframe final deberá venir con el valor original de trade_details.
* Hacerlo de la manera más eficiente posible a nivel computacional.

### Inicialización de SparkSession:

In [1]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder()
                        .appName("Reto 1")
                        .master("local")
                        .getOrCreate()

Intitializing Scala interpreter ...

Spark Web UI available at http://L2111011.bosonit.local:4041
SparkContext available as 'sc' (version = 3.1.2, master = local[*], app id = local-1641372883567)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@1d1b8c


### Carga de CSV

In [2]:
val trade_details = spark.read.format("csv")
                              .option("header", "true")
                              .option("delimiter", ";")
                              .load("C:/Users/sara.arribas/Documents/Big Data/EjerciciosPostSpark/Spark for Data Engineers 1/reto1/trade_details.csv")

val trade_details_snapshot = spark.read.format("csv")
                                       .option("header", "true")
                                       .option("delimiter", ";")
                                       .load("C:/Users/sara.arribas/Documents/Big Data/EjerciciosPostSpark/Spark for Data Engineers 1/reto1/trade_details_snapshot.csv")

trade_details: org.apache.spark.sql.DataFrame = [mfamily: string, mgroup: string ... 4 more fields]
trade_details_snapshot: org.apache.spark.sql.DataFrame = [mfamily: string, mgroup: string ... 4 more fields]


In [197]:
trade_details.sort($"origin_contract_number", $"maturity".desc).show(50)
trade_details_snapshot.orderBy($"origin_contract_number", $"maturity".desc).show(50)

+-------+------+-----+-------------------+----------------------+----------+
|mfamily|mgroup|mtype|origin_trade_number|origin_contract_number|  maturity|
+-------+------+-----+-------------------+----------------------+----------+
|    EQD| EQUIT|  FWD|           10000001|              10000001|2019-07-02|
|    IRD|  BOND|  FWD|           10000009|              10000009|2021-06-12|
|    IRD|   IRS| null|          111222333|             111222333|2024-10-15|
|    IRD| LN_BR| null|           14596583|              13774383|2020-12-29|
|    IRD|   IRS| null|           18343978|              17356077|2024-10-15|
|    IRD| LN_BR| null|           19203839|              18176215|2022-10-06|
|   CURR|   FXD|SWLEG|           19665186|              18622136|2020-12-30|
|   CURR|   FXD|SWLEG|           19665185|              18622136|2020-04-29|
|   CURR|   FXD|SWLEG|           19772399|              18724280|      NULL|
|   CURR|   FXD|SWLEG|           19772400|              18724280|2021-11-05|

### Resultado:

<div class="alert alert-block alert-success">
<b>PASOS A SEGUIR:</b>
</div>

<div class="alert alert-block alert-info">

<b>PASO 1</b> 
    
Crear DataFrame en el que la columna <code>"maturity"</code> no tenga <code>"NULL"</code>. Para ello  sustituimos los valores <code>"NULL"</code> por los valores de la columna <code>"maturity"</code> del DataFrame <code>trade_details_snapshot</code>.
    
</div>

<div class="alert alert-block alert-info">

    
<b>PASO 2</b> 
    
En el caso de repeticiones de la columna <code>"origin_contract_number"</code> nos quedamos con la columna <code>"origin_contract_number"</code> con fecha más actual en la columna <code>"maturity"</code>.


</div>

<div class="alert alert-block alert-info">
   
<b>PASO 3</b>

La columna <code>"maturity"</code> debe estar como el original (con sus <code>NULL</code>).


</div>

**INSTRUCCIONES**: El DataFrame resultante debe almacenarse en la variable `resultado`, sustituyendo el valor `None` por el código que consideréis oportuno. De esta forma podréis comprobar si el resultado es correcto.


In [93]:
val columna = trade_details.where(col("maturity") === "NULL")
columna.show()

+-------+------+-----+-------------------+----------------------+--------+
|mfamily|mgroup|mtype|origin_trade_number|origin_contract_number|maturity|
+-------+------+-----+-------------------+----------------------+--------+
|    SCF|   SCF|  SCF|            3815982|               3672136|    NULL|
|    IRD|   IRS| null|          444111222|             444111222|    NULL|
|    IRD|   IRS| null|          555111222|             555111222|    NULL|
|    IRD|   IRS| null|          556111214|             556111214|    NULL|
|   CURR|   FXD|SWLEG|           19772399|              18724280|    NULL|
|   CURR|   FXD|SWLEG|           20980932|              19883451|    NULL|
|   CURR|   FXD|SWLEG|           20980933|              19883451|    NULL|
|   CURR|   FXD|SWLEG|           22798004|              21622649|    NULL|
|   CURR|   FXD|SWLEG|           22798005|              21622649|    NULL|
+-------+------+-----+-------------------+----------------------+--------+



columna: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [mfamily: string, mgroup: string ... 4 more fields]


In [221]:

val resultado = trade_details.alias("fst").join( trade_details_snapshot.alias("snd"),Seq("origin_contract_number"),"outer")
                                        .where(col("fst.maturity").isNotNull)

// Al crear la columna maturity_noNull sustituimos valores nulos por los de trade_details_snapshot
                                        .withColumn("maturity_noNull", when(col("fst.maturity") === "NULL",col("snd.maturity"))
                                                                       .otherwise(col("fst.maturity")))
// Al crear la columna maturity_Null guardamos valores nulos
                                        .withColumn("maturity_Null", when(col("fst.maturity")==="NULL","NULL").otherwise("0"))
// Usamos un select por la ambigÜedad
                                        .select("origin_contract_number","fst.*","maturity_noNull","maturity_Null")
// Agrupamos de manera que nos queda el máximo de la columna maturity, es decir, la long_leg
                                        .groupBy("mfamily", "mgroup", "mtype","origin_trade_number","origin_contract_number")
                                        .agg(max("maturity_noNull").alias("maturity"), max("maturity_Null").alias("Nulls"))
                                        .orderBy(desc("maturity"))
// cogemos el primer origin_trade_number correspondiente a maturity deseada
                                        .groupBy("mfamily", "mgroup", "mtype","origin_contract_number", "Nulls")
                                        .agg(first("origin_trade_number").alias("origin_trade_number"),
                                             first("maturity").alias("maturity"))
// sustituimos de vuelta los NULL donde estaban
                                        .withColumn("maturity", when(col("Nulls")==="NULL", "NULL").otherwise(col("maturity")))
                                        .drop("Nulls")
                                        .orderBy(desc("maturity"))
                                        .groupBy("mfamily", "mgroup", "mtype","origin_contract_number")
                                        .agg(first("origin_trade_number").alias("origin_trade_number"),
                                             first("maturity").alias("maturity"))


resultado.sort("origin_contract_number").show(50)

+-------+------+-----+----------------------+-------------------+----------+
|mfamily|mgroup|mtype|origin_contract_number|origin_trade_number|  maturity|
+-------+------+-----+----------------------+-------------------+----------+
|    EQD| EQUIT|  FWD|              10000001|           10000001|2019-07-02|
|    IRD|  BOND|  FWD|              10000009|           10000009|2021-06-12|
|    IRD|   IRS| null|             111222333|          111222333|2024-10-15|
|    IRD| LN_BR| null|              13774383|           14596583|2020-12-29|
|    IRD|   IRS| null|              17356077|           18343978|2024-10-15|
|    IRD| LN_BR| null|              18176215|           19203839|2022-10-06|
|   CURR|   FXD|SWLEG|              18622136|           19665186|2020-12-30|
|   CURR|   FXD|SWLEG|              18724280|           19772399|      NULL|
|    IRD|    CF| null|              19433281|           20513130|2021-07-06|
|    IRD|    CF| null|              19453781|           20533916|2023-07-06|

resultado: org.apache.spark.sql.DataFrame = [mfamily: string, mgroup: string ... 4 more fields]


Ejecuta la siguiente celda (no modifiques su código) y te dirá si tu solución es correcta o no. En caso de ser correcta, se ejecutará correctamente y no mostrará nada, pero si no lo es mostrará un error. Además de esas pruebas, se realizarán algunas más (ocultas) a la hora de puntuar el ejercicio, pero evaluar dicha celda es un indicador bastante fiable acerca de si realmente has implementado la solución correcta o no.

In [222]:
assert(resultado.count() == 26)
assert(resultado.orderBy("origin_contract_number").collect()(24)(4) == "564367838")
assert(resultado.orderBy("origin_contract_number").collect()(19)(5) == "NULL")
assert(resultado.orderBy("origin_trade_number").collect()(16)(5) == "NULL")